# Brightway Introductory Session

Welcome! In this session, we will learn the fundamentals of performing a Life Cycle Assessment (LCA) using the Brightway2 framework.

## Part 1: Setup and Finding an Activity

First, we import the `brightway2` library. Then, we connect to the project and database we created during the pre-work setup.

In [ ]:
import brightway2 as bw

# Connect to the project we created with the import script
bw.projects.set_current('bw-course-project')

# Get a reference to our imported Ecoinvent database
db = bw.Database('ecoinvent-3.11-cutoff')

Let's search for the `'water pump operation, electric'` process located in Spain (`ES`).

In [ ]:
# Search for activities containing 'water pump operation, electric'
search_results = db.search('water pump operation, electric')

# We want the specific one for Spain ('ES')
water_pump_es = None
for activity in search_results:
    if activity['location'] == 'ES':
        water_pump_es = activity
        break

water_pump_es

## Part 2: Performing a Simple LCA

To run an LCA, we need an **activity**, a **functional unit**, and an **impact method**.

In [ ]:
# Define the functional unit: 1 unit of our water pump activity
functional_unit = {water_pump_es: 1}

# Let's choose an impact method. We'll use the IPCC 2021 method for Global Warming Potential (GWP100).
# First, find the method tuple by searching for it.
method_key = None
for method in bw.methods:
    if 'IPCC 2021' in str(method) and 'GWP 100a' in str(method):
        method_key = method
        break

method_key

In [ ]:
# Create the LCA object
lca = bw.LCA(functional_unit, method_key)

# Build the Life Cycle Inventory (LCI)
lca.lci()

# Run the Life Cycle Impact Assessment (LCIA)
lca.lcia()

# Print the score
print(f"LCA Score for '{water_pump_es['name']}' in {water_pump_es['location']}:")
print(f"{lca.score:.4f} {bw.methods.get(method_key).get('unit')}")

## Part 3: Exploring a Unit Process (Technosphere)

What are the direct inputs to our process? We can inspect its `technosphere` exchanges.

In [ ]:
print(f"Inputs for '{water_pump_es['name']}':")
for exchange in water_pump_es.technosphere():
    # .input gives the activity that is providing the input
    # .amount is how much of that input is used
    print(f"- Input: {exchange.input['name']}")
    print(f"  Amount: {exchange.amount:.4f} {exchange.input['unit']}")
    print(f"  Location: {exchange.input['location']}")
    print("-" * 20)

## BONUS: Creating a New Process by Changing Geographies

Let's create a new 'water pump' process, but this time, we'll imagine it's running in France (`FR`). We need to copy the original process and swap its electricity input from the Spanish market to the French market.

In [ ]:
# First, find the French low voltage electricity market
electricity_fr = db.get('market for electricity, low voltage', location='FR')

# Create a copy of our Spanish process
# We need a new, unique code for our new activity. We can make one up.
new_code = 'water-pump-fr-electricity'
water_pump_fr = water_pump_es.copy(code=new_code)

# Change its name to avoid confusion
water_pump_fr['name'] = 'water pump operation, electric, FR electricity'
water_pump_fr.save()

Now we find the electricity exchange in our new copied process and change its `input` to be the French electricity we found.

In [ ]:
# Iterate through the exchanges of our NEW French pump process
for exchange in water_pump_fr.technosphere():
    # Find the electricity input
    if 'electricity, low voltage' in exchange.input['name']:
        # Change the input of this exchange to the French electricity market
        exchange['input'] = electricity_fr
        # Save the change
        exchange.save()
        print("Electricity input swapped to FR market!")

Finally, let's run the LCA on our new process and compare the scores!

In [ ]:
# Run the LCA for the new French-powered pump
lca_fr = bw.LCA({water_pump_fr: 1}, method_key)
lca_fr.lci()
lca_fr.lcia()

print("--- Comparative LCA Results ---")
print(f"Original (ES electricity): {lca.score:.4f} kg CO2-eq")
print(f"Modified (FR electricity): {lca_fr.score:.4f} kg CO2-eq")